In [3]:
import tkinter as tk
from tkinter import Canvas, Button, simpledialog, messagebox
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image, ImageOps
import requests

# Function to create an interactive drawing canvas and save as a PNG image
def draw_sketch():
    def save_sketch():
        canvas.postscript(file="sketch.eps", colormode='color')
        img = Image.open("sketch.eps")
        img = img.convert("L")
        img = ImageOps.invert(img)
        img.save("sketch.png")
        window.destroy()
        process_and_visualize_sketch("sketch.png")

    window = tk.Tk()
    window.title("Draw your property layout")
    canvas = Canvas(window, width=400, height=400, bg='white')
    canvas.pack()

    def paint(event):
        x1, y1 = (event.x - 2), (event.y - 2)
        x2, y2 = (event.x + 2), (event.y + 2)
        canvas.create_oval(x1, y1, x2, y2, fill='black', width=2)

    canvas.bind("<B1-Motion>", paint)
    button = Button(window, text="Save Sketch", command=save_sketch)
    button.pack()
    window.mainloop()

# Function to process and visualize the sketch, and detect vertices
def process_and_visualize_sketch(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        messagebox.showerror("Error", "Failed to load the sketch image.")
        return

    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        messagebox.showerror("Error", "No contours detected.")
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)

    # Visualize detected vertices and mark them
    plt.figure(figsize=(8, 8))
    plt.imshow(img, cmap='gray')
    approx = approx.squeeze()
    for i, point in enumerate(approx):
        plt.plot(point[0], point[1], 'ro')
        plt.text(point[0], point[1], f'P{i+1}', color='blue', fontsize=12)

    plt.title('Detected Property Layout with Points')
    plt.axis('off')
    plt.show()

    # Ask user for dimensions between points
    dimensions = []
    for i in range(len(approx)):
        next_point = (i + 1) % len(approx)
        dim = simpledialog.askfloat("Input", f"Enter the length between P{i+1} and P{next_point+1} (in meters):")
        if dim is None:
            messagebox.showerror("Error", "Invalid dimension entered.")
            return
        dimensions.append(dim)

    # Scale and visualize the layout using dimensions
    adjust_points_and_visualize(approx, dimensions)

# Function to adjust points based on user dimensions and visualize them
def adjust_points_and_visualize(points, dimensions):
    # Normalize distances between points using user input
    scaled_points = [points[0]]  # Start with the first point
    for i in range(1, len(points)):
        prev_point = scaled_points[-1]
        original_dist = np.linalg.norm(points[i] - points[i - 1])
        scale_factor = dimensions[i - 1] / original_dist
        new_point = prev_point + (points[i] - points[i - 1]) * scale_factor
        scaled_points.append(new_point)

    # Close the shape if necessary
    if not np.array_equal(scaled_points[-1], scaled_points[0]):
        scaled_points.append(scaled_points[0])

    # Calculate area using the Shoelace formula
    area = calculate_polygon_area(scaled_points)

    # Plot scaled layout with area displayed
    plt.figure(figsize=(8, 8))
    for i, point in enumerate(scaled_points):
        next_point = scaled_points[(i + 1) % len(scaled_points)]
        plt.plot([point[0], next_point[0]], [point[1], next_point[1]], 'b-')
        plt.text(point[0], point[1], f'P{i+1}', color='red', fontsize=12)

    plt.title(f'Scaled Property Layout\nCalculated Area: {area:.2f} square meters')
    plt.axis('equal')
    plt.show()

# Function to calculate the area of a polygon using the Shoelace formula
def calculate_polygon_area(points):
    n = len(points)
    area = 0.5 * abs(sum(points[i][0] * points[(i + 1) % n][1] - points[(i + 1) % n][0] * points[i][1] for i in range(n)))
    return area

# Start the drawing process
draw_sketch()